En esta segunda parte del proyecto final tengo como finalidad crear un modelo para saber si el precio de una acción subirá o bajará en función de alguna noticia que haya salido a los medios referente a la empresa de la acción en la cuál estamos interesados.
Para ello, utilizaré como modelo preentrenado el modelo FinBERT, el cuál a grandes rasgos está entrenado para estimar el sentimiento de una noticia. Se modificarán algunas capas de este modelo para en lugar de obtener una lectura del sentimiento sobre una noticia, obtengamos el impacto de una noticia financiera sobre el futuro precio de una acción particular.


In [2]:
# BLOQUE 1: CONFIGURACIÓN INICIAL Y CARGA DE DATOS


# Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")

# Cargamos nuestro dataset de noticias
# NOTA: Ajusta la ruta según dónde descargaste el archivo de Kaggle
df_news = pd.read_csv(r'stocks_dataset.csv.zip')

print(" *Información clave de nuestro dataset")
print("=" * 50)
print(f"Total de filas: {len(df_news):,}")
print(f"Total de columnas: {len(df_news.columns)}")
print(f"\nColumnas disponibles: {list(df_news.columns)}")

# Exploramos un poco nuestro dataset
print("\nPrimeras 5 filas:")
print(df_news.head())

print("\nTipos de datos orgininales:")
print(df_news.dtypes)

# Intentamos conversión de fechas automática por un formato más aceptable y general
df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce', utc=True)

# Verificamos cuántas fechas no se pudieron convertir
fechas_invalidas = df_news['date'].isnull().sum()
print(f"Fechas que no se pudieron convertir: {fechas_invalidas}")

if fechas_invalidas > 0:
    print(f"Se encontraron {fechas_invalidas} fechas inválidas, serán eliminadas")
    df_news = df_news.dropna(subset=['date'])
    print(f"Filas restantes: {len(df_news):,}")

# Convertimos a un formato sin zona horaria para facilitar operaciones
df_news['date'] = df_news['date'].dt.tz_localize(None)

print("Fechas convertidas exitosamente")

# Verificamos valores nulos
print("\nValores nulos por columna:")
print(df_news.isnull().sum())

# Información de acciones
print(f"\n Acciones únicas (empresas) en el dataset: {df_news['stock'].nunique()}")
print(f"\n Top 10 acciones con más noticias:")
print(df_news['stock'].value_counts().head(10))

# Rango temporal
print(f"\nRango temporal:")
print(f"Fecha inicial: {df_news['date'].min()}")
print(f"Fecha final: {df_news['date'].max()}")
print(f"Período total: {(df_news['date'].max() - df_news['date'].min()).days} días")

# Distribución temporal
df_news['año'] = df_news['date'].dt.year
print(f"\n Distribución de noticias por año:")
print(df_news['año'].value_counts().sort_index())

# Guardamos una versión limpia
df_news.to_csv('noticias_limpio.csv', index=False)
print(" Dataset limpio guardado como 'noticias_limpio.csv' ")


 *Información clave de nuestro dataset
Total de filas: 1,400,469
Total de columnas: 4

Columnas disponibles: ['Unnamed: 0', 'title', 'date', 'stock']

Primeras 5 filas:
   Unnamed: 0                                              title  \
0         0.0            Stocks That Hit 52-Week Highs On Friday   
1         1.0         Stocks That Hit 52-Week Highs On Wednesday   
2         2.0                      71 Biggest Movers From Friday   
3         3.0       46 Stocks Moving In Friday's Mid-Day Session   
4         4.0  B of A Securities Maintains Neutral on Agilent...   

                        date stock  
0  2020-06-05 10:30:00-04:00     A  
1  2020-06-03 10:45:00-04:00     A  
2  2020-05-26 04:30:00-04:00     A  
3  2020-05-22 12:45:00-04:00     A  
4  2020-05-22 11:38:00-04:00     A  

Tipos de datos orgininales:
Unnamed: 0    float64
title          object
date           object
stock          object
dtype: object
Fechas que no se pudieron convertir: 2578
Se encontraron 2578 fechas 